In [14]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D
import pickle
import time
from tensorflow.keras.callbacks import TensorBoard
import numpy as np 
import cv2

In [15]:
X_train = pickle.load(open("D:\projects\Scenic images\Xtrain.pickle","rb"))
y_train = pickle.load(open('D:\projects\Scenic images\ytrain.pickle',"rb"))

X_test = pickle.load(open("D:\projects\Scenic images\Xtest.pickle","rb"))
y_test = pickle.load(open('D:\projects\Scenic images\ytest.pickle',"rb"))

In [16]:
X_train = X_train/255.0
X_test = X_test/255.0

In [17]:
layer_size = [32,64,128,256]
NAME = "VGG-32-64-kernel-dropout0.5-0.7-20"
def VGG():
    
    VGG = Sequential([

        Conv2D(layer_size[0],(3,3),(1,1), input_shape=X_train.shape[1:], activation='relu'),
        Conv2D(layer_size[0],(3,3),(1,1),activation='relu'),
        MaxPool2D(pool_size=(2,2),strides=(2,2)),

   
        Conv2D(layer_size[1],(3,3),(1,1), activation='relu'),
        Conv2D(layer_size[1],(3,3),(1,1), activation='relu'),
        MaxPool2D(pool_size=(2,2),strides=(2,2)),

        
        Conv2D(layer_size[2],(3,3),(1,1), activation='relu'),
        Conv2D(layer_size[2],(3,3),(1,1), activation='relu'),
        Conv2D(layer_size[2],(3,3),(1,1), activation='relu'),
        MaxPool2D(pool_size=(2,2),strides=(2,2)),

        
        # Conv2D(layer_size[3],(3,3),(1,1), activation='relu'),
        # Conv2D(layer_size[3],(3,3),(1,1), activation='relu'),
        # Conv2D(layer_size[3],(3,3),(1,1), activation='relu'),
        # MaxPool2D(pool_size=(2,2),strides=(2,2)),

        
        # Conv2D(layer_size[3],(3,3),(1,1), activation='relu'),
        # Conv2D(layer_size[3],(3,3),(1,1), activation='relu'),
        # Conv2D(layer_size[3],(3,3),(1,1), activation='relu'),
        # MaxPool2D(pool_size=(2,2),strides=(2,2)),

        Flatten(),
        Dropout(0.5),
        Dense(1000,activation='relu'),
        Dropout(0.7),
        Dense(1000,activation='relu'),
        Dense(6,activation='softmax'),
    ])

    return VGG
    
model = VGG()

tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'] )
model.summary()

with tf.device('/gpu:0'):
    model.fit(X_train, y_train, batch_size=32, epochs=20, validation_split=0.3, callbacks =[tensorboard])


results = model.evaluate(X_test, y_test, batch_size = 32)
print("Loss,Acc= ",results)



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 148, 148, 32)      320       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 146, 146, 32)      9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 71, 71, 64)        18496     
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 69, 69, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 34, 34, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 32, 32, 128)      

In [18]:
results = model.evaluate(X_test, y_test, batch_size = 32)

94/94 [==============================] - 2s 24ms/step - loss: 0.6174 - accuracy: 0.7897


In [19]:
pred = []
prednew = []
pred = model.predict([X_test],batch_size=32)
for i in range(len(pred)):
    prednew.append(np.argmax(pred[i])) 

In [22]:
#confusion matrix

tf.math.confusion_matrix(y_test, prednew , num_classes=6)

<tf.Tensor: shape=(6, 6), dtype=int32, numpy=
array([[367,   1,   3,   4,  10,  52],
       [  3, 449,   4,   6,   0,  12],
       [ 10,  11, 338,  85,  96,  13],
       [ 18,   7,  70, 358,  71,   1],
       [ 16,   2,  16,  22, 451,   3],
       [ 79,   4,   3,   2,   7, 406]])>

In [25]:
CATEGORIES = ['buildings', 'forest','glacier', 'mountain', 'sea', 'street']
def prepare(filepath):
    SIZE = 150
    img_arr =  cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_arr = cv2.resize(img_arr, (SIZE,SIZE))
    return new_arr.reshape(-1, SIZE,SIZE, 1)

# model = tf.keras.models.load_model("Multiclass.model")
prediction = model.predict([prepare('D:/projects/Scenic images/dataset/seg_pred/seg_pred/347.jpg')])

print(CATEGORIES[np.argmax(prediction)])

buildings
